In [1]:
"""
============
3D animation
============

A simple example of an animated plot... In 3D!
"""
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation


def Gen_RandLine(length, dims=2):
    """
    Create a line using a random walk algorithm

    length is the number of points for the line.
    dims is the number of dimensions the line has.
    """
    lineData = np.empty((dims, length))
    lineData[:, 0] = np.random.rand(dims)
    for index in range(1, length):
        # scaling the random numbers by 0.1 so
        # movement is small compared to position.
        # subtraction by 0.5 is to change the range to [-0.5, 0.5]
        # to allow a line to move backwards.
        step = ((np.random.rand(dims) - 0.5) * 0.1)
        lineData[:, index] = lineData[:, index - 1] + step

    return lineData


def update_lines(num, dataLines, lines):
    for line, data in zip(lines, dataLines):
        # NOTE: there is no .set_data() for 3 dim data...
        line.set_data(data[0:2, :num])
        line.set_3d_properties(data[2, :num])
    return lines

# Attaching 3D axis to the figure
fig = plt.figure()
ax = p3.Axes3D(fig)

# Fifty lines of random 3-D lines
data = [Gen_RandLine(25, 3) for index in range(50)]

# Creating fifty line objects.
# NOTE: Can't pass empty arrays into 3d version of plot()
lines = [ax.plot(dat[0, 0:1], dat[1, 0:1], dat[2, 0:1])[0] for dat in data]

# Setting the axes properties
ax.set_xlim3d([0.0, 1.0])
ax.set_xlabel('X')

ax.set_ylim3d([0.0, 1.0])
ax.set_ylabel('Y')

ax.set_zlim3d([0.0, 1.0])
ax.set_zlabel('Z')

ax.set_title('3D Test')

# Creating the Animation object
line_ani = animation.FuncAnimation(fig, update_lines, 25, fargs=(data, lines),
                                   interval=50, blit=False)

plt.show()

In [3]:
#!/usr/bin/python
#####################################################################
## Cube animation program using python, numpy and matplotlib.  In the
## spirit of old-school demo programs... This progam can teach you one
## thing or two about computer graphics, specially how important it is
## to learn linear algebra.
##
## This also shows a way to create animations using matplotlib. Just
## make a loop that plots each new frame, and save that frame to an
## image. You can then make a film from these images using mplayer or
## whatever. Instead of replotting everything you could just change
## the parameters from the image curves, but in this case it didn't
## seem much of an advantage to do that.
##
## You can see the output on YouTube:
##   http://www.youtube.com/watch?v=K5qoS781qOw
##
## Coded by Nicolau Werneck<nwerneck@gmail.com> in 2011-03-19
##
#####################################################################

from pylab import *
import numpy as np


#####################################################################
## Produces a rotation matrix from the 3 last components of a
## quaternion.
def quaternion_to_matrix(myx):
    # '''converts from a quaternion representation (last 3 values) to rotation matrix.'''
    xb,xc,xd = myx

    xnormsq = xb*xb+xc*xc+xd*xd

    if xnormsq < 1:
        ## If inside the unit sphere, these are the components
        ## themselves, and we just have to calculate a to normalize
        ## the quaternion.
        b,c,d = xb,xc,xd
        a = np.sqrt(1-xnormsq)
    else:
        ## Just to work gracefully if we have invalid inputs, we
        ## reflect the vectors outside the unit sphere to the other
        ## side, and use the inverse norm and negative values of a.
        b,c,d = -xb/xnormsq,-xc/xnormsq,-xd/xnormsq
        a = -np.sqrt(1-  1.0/xnormsq  )
        ## This should not be used, it defeats the whole concept that
        ## small (b,c,d) vector norms have small rotation angles. It's
        ## really just to let us work near the borders of the
        ## sphere. Any optimization algorithm should work with
        ## initalizations just inside the spere, and avoid wandering
        ## outside of it.

    assert a >= -1
    assert a <= 1

    ## Notice we return a transpose matrix, because we work with line-vectors

    return np.array([ [(a*a+b*b-c*c-d*d), (2*b*c-2*a*d),     (2*b*d+2*a*c)      ],
                         [(2*b*c+2*a*d),     (a*a-b*b+c*c-d*d), (2*c*d-2*a*b)      ],
                         [(2*b*d-2*a*c),     (2*c*d+2*a*b),     (a*a-b*b-c*c+d*d)] ]  ).T  \
                         / (a*a+b*b+c*c+d*d)
                         
##
#####################################################################


## Calculate quaternion values using sinusoids over the frame index
## number, and get rotation matrix. Notice that the second parameter
## is the number of frames. That allows us to easily create an
## animation that can be played as a loop.
def crazy_rotation(ind,Nind):
    return quaternion_to_matrix(0.5*sin(pi*2*ind*Nind**-1.*array([1,2,3])))

## This function calculate the projected image coordinates form the 3D
## coordinates of the object vertices.
def project(D, vecs):
    return vvs[:,:2]/(vvs[:,[2,2]]-D)

## The cube vertices
vs = reshape(mgrid[-1:2:2,-1:2:2,-1:2:2].T, (8,3))

## Generate the list of connected vertices
ed=[(j,k)
    for j in range(8)
    for k in range(j,8)
    if sum(abs(vs[j]-vs[k]))==2 ]


## Camera position
D=-5

## Create the figure. figsize needed to set the image sizes at the end...
figure(1, figsize=(6.4,4.8))

## create/get the axes object.
ax=subplot(1,1,1)

## Set image title.
suptitle('Cube animation')

## Number of frames.
Nind=250

##
## Now start a loop over the animation frames. The index is used both
## to name the images and to calculate the rotation matrix. You could
## use this index to make some physics stuff...
for ind in range(Nind):
    print ind
    ## This is crucial, clears the figure for the new plot. 
    ax.clear() 

    ## Get the rotation matrix for the current frame.
    rotM = crazy_rotation(ind, Nind)

    ## Calculate the 3D coordinates of the vertices of the rotated
    ## cube. Just multiply the vectors by the rotation matrix...
    vvs=dot(vs,rotM)

    ## Now calculate the image coordinates of the points.
    pt = project(D,vvs)

    ## Plot the edges.
    for j,k in ed:
        ax.plot(pt[[j,k],0], pt[[j,k],1], 'g-', lw=3)

    ## Plot the vertices.
    ax.plot(pt[:,0], pt[:,1], 'bo')

    ## Set axes limits
    ax.axis('equal')
    ax.axis([-0.5,0.5,-0.5,0.5])

    ## Save the current frame. We need the dpi (along with the figure
    ## size up there) to set the image size.
    savefig('anim%03d.png'%ind, dpi=100)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249


In [1]:
import imageio
import os

png_names = os.listdir('./png')
filenames = map(lambda p: os.path.abspath(os.path.join('./png', p)), png_names)


print filenames[:10]

#print os.path.exists('anim000.png')

#im = imageio.imread('anim000.png')

with imageio.get_writer('cube.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

['/home/criss/tutorial/matplotlib/png/anim234.png', '/home/criss/tutorial/matplotlib/png/anim069.png', '/home/criss/tutorial/matplotlib/png/anim011.png', '/home/criss/tutorial/matplotlib/png/anim243.png', '/home/criss/tutorial/matplotlib/png/anim017.png', '/home/criss/tutorial/matplotlib/png/anim104.png', '/home/criss/tutorial/matplotlib/png/anim122.png', '/home/criss/tutorial/matplotlib/png/anim101.png', '/home/criss/tutorial/matplotlib/png/anim152.png', '/home/criss/tutorial/matplotlib/png/anim132.png']


AttributeError: close

In [8]:
import os, sys

#dataDir = 'png'

#print os.listdir('.')

#os.chdir(dataDir)


os.system('SETLOCAL EnableDelayedExpansion')
#This is the path of the imagemagick installation convert command.  
#The invalid parameter I was  getting was because the computer was trying to 
#use a different convert command.  This code sets convert as a different
#variable and then calls that new variable as the convert command.  
#os.system('SET IMCONV="C:\Program Files\ImageMagick-6.9.1-Q16\Convert"')
#os.system('%IMCONV% *.jpg animated.gif')
os.system('convert png/*.png animated.gif')

0

In [22]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations


fig = plt.figure()
ax = fig.gca(projection='3d')
ax.set_aspect("equal")

# draw cube
r = [-1, 1]
for s, e in combinations(np.array(list(product(r, r, r))), 2):
    if np.sum(np.abs(s-e)) == r[1]-r[0]:
        ax.plot3D(*zip(s, e), color="b")
        
plt.show()

In [20]:
a = np.array([1,2,3])
for s, e in combinations(a, 2):
    print str(s) + ' ' + str(e)

for s, e in combinations(np.array(list(product(r, r, r))), 2):
    print s
    print e
    print np.abs(s-e)
    print np.sum(np.abs(s-e))
    print zip(s,e)

1 2
1 3
2 3
[-1 -1 -1]
[-1 -1  1]
[0 0 2]
2
[(-1, -1), (-1, -1), (-1, 1)]
[-1 -1 -1]
[-1  1 -1]
[0 2 0]
2
[(-1, -1), (-1, 1), (-1, -1)]
[-1 -1 -1]
[-1  1  1]
[0 2 2]
4
[(-1, -1), (-1, 1), (-1, 1)]
[-1 -1 -1]
[ 1 -1 -1]
[2 0 0]
2
[(-1, 1), (-1, -1), (-1, -1)]
[-1 -1 -1]
[ 1 -1  1]
[2 0 2]
4
[(-1, 1), (-1, -1), (-1, 1)]
[-1 -1 -1]
[ 1  1 -1]
[2 2 0]
4
[(-1, 1), (-1, 1), (-1, -1)]
[-1 -1 -1]
[1 1 1]
[2 2 2]
6
[(-1, 1), (-1, 1), (-1, 1)]
[-1 -1  1]
[-1  1 -1]
[0 2 2]
4
[(-1, -1), (-1, 1), (1, -1)]
[-1 -1  1]
[-1  1  1]
[0 2 0]
2
[(-1, -1), (-1, 1), (1, 1)]
[-1 -1  1]
[ 1 -1 -1]
[2 0 2]
4
[(-1, 1), (-1, -1), (1, -1)]
[-1 -1  1]
[ 1 -1  1]
[2 0 0]
2
[(-1, 1), (-1, -1), (1, 1)]
[-1 -1  1]
[ 1  1 -1]
[2 2 2]
6
[(-1, 1), (-1, 1), (1, -1)]
[-1 -1  1]
[1 1 1]
[2 2 0]
4
[(-1, 1), (-1, 1), (1, 1)]
[-1  1 -1]
[-1  1  1]
[0 0 2]
2
[(-1, -1), (1, 1), (-1, 1)]
[-1  1 -1]
[ 1 -1 -1]
[2 2 0]
4
[(-1, 1), (1, -1), (-1, -1)]
[-1  1 -1]
[ 1 -1  1]
[2 2 2]
6
[(-1, 1), (1, -1), (-1, 1)]
[-1  1 -1]
[ 1  1 -1]
[

In [22]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')


def cc(arg):
    return mcolors.colorConverter.to_rgba(arg, alpha=0.6)

xs = np.arange(0, 10, 0.4)
verts = []
zs = [0.0, 1.0, 2.0, 3.0]
for z in zs:
    ys = np.random.rand(len(xs))
    ys[0], ys[-1] = 0, 0
    verts.append(list(zip(xs, ys)))

poly = PolyCollection(verts, facecolors=[cc('r'), cc('g'), cc('b'),
                                         cc('y')])
poly.set_alpha(0.7)
ax.add_collection3d(poly, zs=zs, zdir='y')

ax.set_xlabel('X')
ax.set_xlim3d(0, 10)
ax.set_ylabel('Y')
ax.set_ylim3d(-1, 4)
ax.set_zlabel('Z')
ax.set_zlim3d(0, 1)

plt.show()

In [16]:
xs = np.arange(0, 10, 0.4)
ys = np.random.rand(len(xs))

print len(zip(xs, ys))

25


In [1]:
x = ["abc", "efg"]
print x

['abc', 'efg']